In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../../src')))
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../../scripts')))

In [3]:
from utils import *
from plottings import *

In [26]:
df_test = pd.read_csv('../../data/cleaned_test_data.csv')

In [4]:
df = pd.read_csv('../../data/train.csv')

C:\Users\Admin\AppData\Local\Temp\ipykernel_8296\1363763938.py:1: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('../../data/train.csv')


In [28]:
column_summary(df_test)

,col_name,col_dtype,num_of_nulls,num_of_non_nulls,num_of_distinct_values,distinct_values_counts
0,Id,int64,0,41088,41088,"{1: 1, 27446: 1, 27388: 1, 27389: 1, 27390: 1,..."
1,Store,int64,0,41088,856,"{1: 48, 727: 48, 729: 48, 731: 48, 732: 48, 73..."
2,DayOfWeek,int64,0,41088,7,"{4: 5992, 3: 5992, 2: 5992, 1: 5992, 7: 5992, ..."
3,Date,object,0,41088,48,"{'2015-09-17': 856, '2015-09-16': 856, '2015-0..."
4,Open,float64,0,41088,2,"{1.0: 35104, 0.0: 5984}"
5,Promo,int64,0,41088,2,"{0: 24824, 1: 16264}"
6,StateHoliday,object,0,41088,2,"{'0': 40908, 'a': 180}"
7,SchoolHoliday,int64,0,41088,2,"{0: 22866, 1: 18222}"


In [5]:
column_summary(df)

,col_name,col_dtype,num_of_nulls,num_of_non_nulls,num_of_distinct_values,distinct_values_counts
0,Store,int64,0,1017209,1115,"{1: 942, 726: 942, 708: 942, 709: 942, 713: 94..."
1,DayOfWeek,int64,0,1017209,7,"{5: 145845, 4: 145845, 3: 145665, 2: 145664, 1..."
2,Date,object,0,1017209,942,"{'2015-07-31': 1115, '2013-11-06': 1115, '2013..."
3,Sales,int64,0,1017209,21734,"{0: 172871, 5674: 215, 5558: 197, 5483: 196, 6..."
4,Customers,int64,0,1017209,4086,"{0: 172869, 560: 2414, 576: 2363, 603: 2337, 5..."
5,Open,int64,0,1017209,2,"{1: 844392, 0: 172817}"
6,Promo,int64,0,1017209,2,"{0: 629129, 1: 388080}"
7,StateHoliday,object,0,1017209,5,"{'0': 855087, 0: 131072, 'a': 20260, 'b': 6690..."
8,SchoolHoliday,int64,0,1017209,2,"{0: 835488, 1: 181721}"


In [29]:
df_test.StateHoliday = df_test.StateHoliday.replace({'0': 0})

In [7]:
df.StateHoliday = df.StateHoliday.replace({'0': 0})

In [30]:
df_test['StateHoliday'] = df_test['StateHoliday'].map({0: 0, 'a': 1, 'b': 2, 'c': 3})

In [9]:
df['StateHoliday'] = df['StateHoliday'].map({0: 0, 'a': 1, 'b': 2, 'c': 3})

In [33]:
df_test['Date'] = pd.to_datetime(df_test['Date'])

# Extract year, month, and day into new columns
df_test['Year'] = df_test['Date'].dt.year
df_test['Month'] = df_test['Date'].dt.month
df_test['Day'] = df_test['Date'].dt.day


In [10]:
df['Date'] = pd.to_datetime(df['Date'])

# Extract year, month, and day into new columns
df['Year'] = df['Date'].dt.year
df['Month'] = df['Date'].dt.month
df['Day'] = df['Date'].dt.day


In [34]:
df_test['Is_Weekend'] = df_test['Date'].dt.dayofweek.apply(lambda x: 1 if x >= 5 else 0)

In [11]:
df['Is_Weekend'] = df['Date'].dt.dayofweek.apply(lambda x: 1 if x >= 5 else 0)

In [35]:
df_test['Date'] = pd.to_datetime(df_test['Date'])
# Identify dates with holidays (StateHoliday != 0)
holiday_mask = df_test['StateHoliday'] != 0
holiday_dates = df_test.loc[holiday_mask, 'Date']

# If there are no holidays, the model will break, so ensure there's at least one holiday
if len(holiday_dates) == 0:
    raise ValueError("No holidays found in the dataset!")

# Append the first and last holiday to the beginning and end of the list, to simulate cyclic behavior
holiday_dates = pd.concat([holiday_dates, holiday_dates + pd.DateOffset(years=1)])

# Calculate "Number of days to holiday" (loop back to the next year's holidays if needed)
df_test['Days_to_Holiday'] = df_test['Date'].apply(lambda x: (holiday_dates[holiday_dates >= x] - x).min().days)

# Calculate "Number of days after holiday" (loop back to the previous year's holidays if needed)
df_test['Days_after_Holiday'] = df_test['Date'].apply(lambda x: (x - holiday_dates[holiday_dates <= x]).min().days)

# Display the DataFrame with the new columns
print(df_test[['Date', 'StateHoliday', 'Days_to_Holiday', 'Days_after_Holiday']])

            Date  StateHoliday  Days_to_Holiday  Days_after_Holiday
0     2015-09-17             0              333                33.0
1     2015-09-17             0              333                33.0
2     2015-09-17             0              333                33.0
3     2015-09-17             0              333                33.0
4     2015-09-17             0              333                33.0
...          ...           ...              ...                 ...
41083 2015-08-01             0               14                 NaN
41084 2015-08-01             0               14                 NaN
41085 2015-08-01             0               14                 NaN
41086 2015-08-01             0               14                 NaN
41087 2015-08-01             0               14                 NaN

[41088 rows x 4 columns]


In [13]:
df['Date'] = pd.to_datetime(df['Date'])
# Identify dates with holidays (StateHoliday != 0)
holiday_mask = df['StateHoliday'] != 0
holiday_dates = df.loc[holiday_mask, 'Date']

# If there are no holidays, the model will break, so ensure there's at least one holiday
if len(holiday_dates) == 0:
    raise ValueError("No holidays found in the dataset!")

# Append the first and last holiday to the beginning and end of the list, to simulate cyclic behavior
holiday_dates = pd.concat([holiday_dates, holiday_dates + pd.DateOffset(years=1)])

# Calculate "Number of days to holiday" (loop back to the next year's holidays if needed)
df['Days_to_Holiday'] = df['Date'].apply(lambda x: (holiday_dates[holiday_dates >= x] - x).min().days)

# Calculate "Number of days after holiday" (loop back to the previous year's holidays if needed)
df['Days_after_Holiday'] = df['Date'].apply(lambda x: (x - holiday_dates[holiday_dates <= x]).min().days)

# Display the DataFrame with the new columns
print(df[['Date', 'StateHoliday', 'Days_to_Holiday', 'Days_after_Holiday']])

              Date  StateHoliday  Days_to_Holiday  Days_after_Holiday
0       2015-07-31             0               64                  42
1       2015-07-31             0               64                  42
2       2015-07-31             0               64                  42
3       2015-07-31             0               64                  42
4       2015-07-31             0               64                  42
...            ...           ...              ...                 ...
1017204 2013-01-01             1                0                   0
1017205 2013-01-01             1                0                   0
1017206 2013-01-01             1                0                   0
1017207 2013-01-01             1                0                   0
1017208 2013-01-01             1                0                   0

[1017209 rows x 4 columns]


In [36]:
def categorize_day(day):
    if day >= 1 and day <= 10:
        return 0  # Beginning of the Month
    elif day >= 11 and day <= 20:
        return 1  # Mid-Month
    else:
        return 2  # End of the Month

# Apply the function to create a new column
df_test['MonthPhase'] = df_test['Day'].apply(categorize_day)


In [17]:
def categorize_day(day):
    if day >= 1 and day <= 10:
        return 0  # Beginning of the Month
    elif day >= 11 and day <= 20:
        return 1  # Mid-Month
    else:
        return 2  # End of the Month

# Apply the function to create a new column
df['MonthPhase'] = df['Day'].apply(categorize_day)


In [37]:
df_test.fillna(-1, inplace=True)

In [39]:
save_dataframe_to_csv(df_test, 'final_test.csv')

In [18]:

save_dataframe_to_csv(df, 'train_final.csv')

In [19]:
column_summary(df)

,col_name,col_dtype,num_of_nulls,num_of_non_nulls,num_of_distinct_values,distinct_values_counts
0,Store,int64,0,1017209,1115,"{1: 942, 726: 942, 708: 942, 709: 942, 713: 94..."
1,DayOfWeek,int64,0,1017209,7,"{5: 145845, 4: 145845, 3: 145665, 2: 145664, 1..."
2,Date,datetime64[ns],0,1017209,942,"{2015-07-31 00:00:00: 1115, 2013-11-06 00:00:0..."
3,Sales,int64,0,1017209,21734,"{0: 172871, 5674: 215, 5558: 197, 5483: 196, 6..."
4,Customers,int64,0,1017209,4086,"{0: 172869, 560: 2414, 576: 2363, 603: 2337, 5..."
5,Open,int64,0,1017209,2,"{1: 844392, 0: 172817}"
6,Promo,int64,0,1017209,2,"{0: 629129, 1: 388080}"
7,StateHoliday,int64,0,1017209,4,"{0: 986159, 1: 20260, 2: 6690, 3: 4100}"
8,SchoolHoliday,int64,0,1017209,2,"{0: 835488, 1: 181721}"
9,Year,int32,0,1017209,3,"{2013: 406974, 2014: 373855, 2015: 236380}"
